# Raw Data

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

path = 'Data/DAC_NationalDownloadableFile.csv'
RawData = pd.read_csv(path, low_memory = False, encoding = 'ISO-8859-1')
RawData.columns = [i.strip() for i in RawData.columns]
cols = ['NPI','frst_nm',  'mid_nm', 'lst_nm', 'suff', 'gndr', 'Cred', 'pri_spec', 'cty', 'st', 'zip']

RawData = RawData[cols]
# RawData = RawData.drop_duplicates().reset_index(drop = True)
RawData

In [ ]:
# NPI and its accounts
print('Unique NPI we have: {}'.format(len(RawData['NPI'].value_counts())))
RawData['NPI'].value_counts()

In [ ]:
# RawData['Cred'].value_counts()#.sum()
UniqueRawData = RawData.groupby('NPI').first().reset_index()
UniqueRawData

In [ ]:
UniqueRawData.fillna('Missing')['Cred'].value_counts()

# Generate Keywords

In [ ]:
def get_doctor_info(DoctorDF):
    doctor_infolist = []
    for _, row in DoctorDF.iterrows():
        # most results: fistname is before lastname
        if row['mid_nm'] == "NaN":
            doctor_info = "{} {} {} {} {}".format(row['frst_nm'], row['lst_nm'], 
                                                  row['pri_spec'], 
                                                  row['cty'], row['st'],
                                                  # row['zip']
                                                    )
        else:
            doctor_info = "{} {} {} {} {} {}".format(row['frst_nm'], row['mid_nm'],  row['lst_nm'], 
                                                  row['pri_spec'], 
                                                  row['cty'], row['st'],
                                                  # row['zip']
                                                )
        # print(doctor_info)
        doctor_infolist.append([row['NPI'], doctor_info])

    return doctor_infolist

In [ ]:
RawData2 = RawData[RawData['Cred'] == 'MD'].reset_index(drop = True)
MDData = RawData2.groupby('NPI').first().reset_index()
MDData = MDData.fillna('NaN')
MDData.to_pickle('Output/MDData.p')
MDData['keyword'] = get_doctor_info(MDData)
MDData[['NPI', 'keyword']].to_pickle('Output/MDDataKeywords.p')
print(MDData.shape)

In [ ]:
RawData2 = RawData[RawData['Cred'].isna()].reset_index(drop = True)
MDData = RawData2.groupby('NPI').first().reset_index()
MDData = MDData.fillna('NaN')
MDData.to_pickle('Output/MissingData.p')
MDData['keyword'] = get_doctor_info(MDData)
MDData[['NPI', 'keyword']].to_pickle('Output/MissingDataKeywords.p')
print(MDData.shape)

In [ ]:
MDData[['NPI', 'keyword']].iloc[:int(len(MDData)/2), ].to_pickle('Output/MissingDataKeywordsPart1.p')
MDData[['NPI', 'keyword']].iloc[int(len(MDData)/2):, ].reset_index(drop = True).to_pickle('Output/MissingDataKeywordsPart2.p')

# Split Half and Half

In [ ]:
old_md_doc_dir = 'Data/MD_DocListDF_All.p'

Old_MD_Doc = pd.read_pickle(old_md_doc_dir)
existed_npi = Old_MD_Doc.iloc[:100000]['NPI'].unique()
len(existed_npi)

In [ ]:
Second_MDData = MDData[-MDData['NPI'].isin(existed_npi)].reset_index(drop = True)
Second_MDData

In [ ]:
Second_MDData[['NPI', 'keyword']].to_pickle('Output/Second_MDDataKeywords.p')